In [1]:
import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
#from glmnet import glmnet; from glmnetPlot import glmnetPlot 
#from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
#from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
#from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict
import glob
import csv
import numpy as np
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
files = glob.glob("final_features/*")
print(len(files))

102


In [3]:
print(files)

['final_features/TCGA-25-1878.csv', 'final_features/TCGA-23-2643.csv', 'final_features/TCGA-23-1809.csv', 'final_features/TCGA-25-1313.csv', 'final_features/TCGA-23-1028.csv', 'final_features/TCGA-23-2079.csv', 'final_features/TCGA-23-2078.csv', 'final_features/TCGA-23-1029.csv', 'final_features/TCGA-25-1312.csv', 'final_features/TCGA-5X-AA5U.csv', 'final_features/TCGA-25-2396.csv', 'final_features/TCGA-23-2084.csv', 'final_features/TCGA-23-2641.csv', 'final_features/TCGA-25-2397.csv', 'final_features/TCGA-25-2393.csv', 'final_features/TCGA-23-2645.csv', 'final_features/TCGA-25-1329.csv', 'final_features/TCGA-25-1315.csv', 'final_features/TCGA-23-2081.csv', 'final_features/TCGA-25-1314.csv', 'final_features/TCGA-25-1328.csv', 'final_features/TCGA-25-2392.csv', 'final_features/TCGA-25-2409.csv', 'final_features/TCGA-25-1316.csv', 'final_features/TCGA-25-1317.csv', 'final_features/TCGA-25-2408.csv', 'final_features/TCGA-23-2647.csv', 'final_features/TCGA-25-2391.csv', 'final_features/TCG

In [4]:
for file in files:
    print(file)

final_features/TCGA-25-1878.csv
final_features/TCGA-23-2643.csv
final_features/TCGA-23-1809.csv
final_features/TCGA-25-1313.csv
final_features/TCGA-23-1028.csv
final_features/TCGA-23-2079.csv
final_features/TCGA-23-2078.csv
final_features/TCGA-23-1029.csv
final_features/TCGA-25-1312.csv
final_features/TCGA-5X-AA5U.csv
final_features/TCGA-25-2396.csv
final_features/TCGA-23-2084.csv
final_features/TCGA-23-2641.csv
final_features/TCGA-25-2397.csv
final_features/TCGA-25-2393.csv
final_features/TCGA-23-2645.csv
final_features/TCGA-25-1329.csv
final_features/TCGA-25-1315.csv
final_features/TCGA-23-2081.csv
final_features/TCGA-25-1314.csv
final_features/TCGA-25-1328.csv
final_features/TCGA-25-2392.csv
final_features/TCGA-25-2409.csv
final_features/TCGA-25-1316.csv
final_features/TCGA-25-1317.csv
final_features/TCGA-25-2408.csv
final_features/TCGA-23-2647.csv
final_features/TCGA-25-2391.csv
final_features/TCGA-25-1628.csv
final_features/TCGA-23-1117.csv
final_features/TCGA-57-1582.csv
final_fe

In [10]:
files[90]

'final_features/TCGA-25-1321.csv'

In [ ]:
features = []
for file in files:
    flag = -1
    file_features = []
    with open(file, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if flag == -1:
                array = row
                for index in range(0, len(array)):
                    file_features.append(float(array[index]))
    features.append(file_features)
print(len(features))

In [ ]:
train_features = []
val_features = []
for index in range(0, 51):
    current_features = []
    current_features.append(index+1)
    for index1 in range(0, len(features[index])):
        current_features.append(features[index][index1])
    train_features.append(current_features)

for index in range(51, len(features)):
    current_features = []
    current_features.append(index+1-51)
    for index1 in range(0, len(features[index])):
        current_features.append(features[index][index1])
    val_features.append(current_features)

In [ ]:
print(len(train_features))
print(len(val_features))

In [ ]:
rows = []
rows.append("")
for index in range(1, 893):
    rows.append("F_" + str(index))
print(len(rows))

In [ ]:
final_train_features = []
final_train_features.append(rows)
for index in range(0, len(train_features)):
    final_train_features.append(train_features[index])
print(len(final_train_features))

In [ ]:
final_val_features = []
final_val_features.append(rows)
for index in range(0, len(val_features)):
    final_val_features.append(val_features[index])
print(len(final_val_features))

In [ ]:
# write csv file
with open("train_features.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_train_features)):
        spamwriter.writerow(final_train_features[index])

In [ ]:
# write csv file
with open("val_features.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_val_features)):
        spamwriter.writerow(final_val_features[index])

In [ ]:
flag = -1
new_filenames = []
survival_info = []
with open("Ov_TCGA_data.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        file_survival_info = []
        if flag == -1:
            flag = 1
        else:
            array = row
            new_filenames.append(array[1])
            if array[10] == "--":
                if array[len(array)-1] == "--":
                    array[len(array)-1] = 50
                file_survival_info.append(float(array[len(array)-1]))
                file_survival_info.append(0)
            else:
                file_survival_info.append(float(array[10]))
                file_survival_info.append(1)
            survival_info.append(file_survival_info)

In [ ]:
y = []
for file in files:
    file = file.split("/")[-1][:-4]
    count = 0
    flag = -1
    for filename in new_filenames:
        if file == filename:
            y.append(survival_info[count])
            flag = 1
        count += 1

In [ ]:
train_labels = []
val_labels = []
for index in range(0, 51):
    current_labels = []
    current_labels.append(index+1)
    for index1 in range(0, len(y[index])):
        current_labels.append(y[index][index1])
    train_labels.append(current_labels)
    
for index in range(51, len(y)):
    current_labels = []
    current_labels.append(index+1-51)
    for index1 in range(0, len(y[index])):
        current_labels.append(y[index][index1])
    val_labels.append(current_labels)
print(len(train_labels))
print(len(val_labels))

In [ ]:
rows = []
rows.append("")
rows.append("Survival (in days)")
rows.append("Status")

In [ ]:
final_train_labels = []
final_train_labels.append(rows)
for index in range(0, len(train_labels)):
    final_train_labels.append(train_labels[index])
print(len(final_train_labels))

In [ ]:
final_val_labels = []
final_val_labels.append(rows)
for index in range(0, len(val_labels)):
    final_val_labels.append(val_labels[index])
print(len(final_val_labels))

In [ ]:
# write csv file
with open("train_labels.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_train_labels)):
        spamwriter.writerow(final_train_labels[index])

In [ ]:
# write csv file
with open("val_labels.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(final_val_labels)):
        spamwriter.writerow(final_val_labels[index])

In [ ]:
rows = []
rows.append("")
rows.append("Filenames")

In [ ]:
filenames = []
filenames.append(rows)
for index in range(0, len(files)):
    f = []
    f.append(index+1)
    f.append(files[index])
    filenames.append(f)

In [ ]:
# write csv file
with open("filenames.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for index in range(0, len(filenames)):
        spamwriter.writerow(filenames[index])

In [ ]:
print(y)

In [ ]:
x = features

In [ ]:
print(x)

In [ ]:
print(np.array(x).shape)
print(np.array(y).shape)

In [ ]:
y_status = []
y_survival = []
for index in range(0, len(y)):
    if y[index][1] == 0:
        y_status.append(False)
    else:
        y_status.append(True)
    y_survival.append(y[index][0])
print(len(y_survival))
print(len(y_status))

In [ ]:
time, survival_prob = kaplan_meier_estimator(y_status, y_survival)
plt.step(time, survival_prob, where="post")
plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")

In [ ]:
estimator = CoxPHSurvivalAnalysis()

In [ ]:
data_x_numeric = OneHotEncoder().fit_transform(x)